# imports/setup

In [21]:
import datetime
import json
import pandas as pd
from collections import defaultdict
from typing import List, Dict, Any
from datamodel import TradingState, Listing, OrderDepth, Trade, Observation

# backtester

In [22]:
# edit to mark to mm mid 

In [23]:
class Backtester:
    def __init__(self, trader, listings: Dict[str, Listing], position_limit: Dict[str, int], fair_marks, 
                 market_data: pd.DataFrame, trade_history: pd.DataFrame, file_name: str = None):
        self.trader = trader
        self.listings = listings
        self.market_data = market_data
        self.position_limit = position_limit
        self.fair_marks = fair_marks
        self.trade_history = trade_history.sort_values(by=['timestamp', 'symbol'])
        self.file_name = file_name

        self.observations = [Observation({}, {}) for _ in range(len(market_data))]

        self.current_position = {product: 0 for product in self.listings.keys()}
        self.pnl_history = []
        self.pnl = {product: 0 for product in self.listings.keys()}
        self.cash = {product: 0 for product in self.listings.keys()}
        self.trades = []
        self.sandbox_logs = []
        
    def run(self):
        traderData = ""
        
        timestamp_group_md = self.market_data.groupby('timestamp') # Generates a dataframe for each instant
        timestamp_group_th = self.trade_history.groupby('timestamp')
        
        own_trades = defaultdict(list) # Declare variable to list
        market_trades = defaultdict(list) # Declare variable to list
        pnl_product = defaultdict(float) # Declare variable to float
        
        trade_history_dict = {} # Gonna be used to organize the history of trades 
        
        for timestamp, group in timestamp_group_th: # Looping through each timestamp in the trade history
            trades = []
            for _, row in group.iterrows(): # Looping through each trade
                
                # Defines the agents of the trade
                symbol = row['symbol']
                price = row['price']
                quantity = row['quantity']
                buyer = row['buyer'] if pd.notnull(row['buyer']) else ""
                seller = row['seller'] if pd.notnull(row['seller']) else ""

                
                trade = Trade(symbol, int(price), int(quantity), buyer, seller, timestamp) # Defines an instance of the Trade class
                
                trades.append(trade) # Adds it to the trades list
            trade_history_dict[timestamp] = trades # Creates an instance for each timestamp (the list of trades instances)
        
        
        for timestamp, group in timestamp_group_md:
            order_depths = self._construct_order_depths(group) # Creates the dictionary of order depths for each product geiven a timestamp
            order_depths_matching = self._construct_order_depths(group) # Idem
            order_depths_pnl = self._construct_order_depths(group) # Idem
            state = self._construct_trading_state(traderData, timestamp, self.listings, order_depths, 
                                 dict(own_trades), dict(market_trades), self.current_position, self.observations) # Creates the trading state for given a timestamp
            orders, conversions, traderData = self.trader.run(state) # Defines the orders to place in the next timestamp according to the strategy developed
            products = group['product'].tolist() # Creates a a list of products
            sandboxLog = "" # Initializes sandboxLog 
            trades_at_timestamp = trade_history_dict.get(timestamp, []) # Gets the trades performed at a given timestamp

            for product in products: # Loops through the products
                new_trades = [] # Creates an empty list that will we filled out with the new trades performed 
                for order in orders.get(product, []): # Loops through the orders defined for each product
                    trades_done, sandboxLog = self._execute_order(timestamp, order, order_depths_matching, self.current_position, self.cash, trade_history_dict, sandboxLog) # Update the trades performed list, both considering the 
                    new_trades.extend(trades_done) # Add the trades done to the products trades at the timestamp
                if len(new_trades) > 0:
                    own_trades[product] = new_trades # Define the trades done of the product at the timestamp

            self.sandbox_logs.append({"sandboxLog": sandboxLog, "lambdaLog": "", "timestamp": timestamp}) # If position limits were tried to be surpassed, record logs

            trades_at_timestamp = trade_history_dict.get(timestamp, []) # Redefine the variable as it has been modifyed
            if trades_at_timestamp:
                for trade in trades_at_timestamp:
                    product = trade.symbol
                    market_trades[product].append(trade) # The dictionary that contains every trade that was performed in the market at that timestamp (including our trades?)
            else: 
                for product in products:
                    market_trades[product] = [] # No trades incolcing this product were done at this timestamp

            
            for product in products:
                self._mark_pnl(self.cash, self.current_position, order_depths_pnl, self.pnl, product)
                self.pnl_history.append(self.pnl[product]) # Updates pnl
            self._add_trades(own_trades, market_trades)
        return self._log_trades(self.file_name)
    
    
    def _log_trades(self, filename: str = None): # Creates the file that gathers the information of the backtesting process
        if filename is None:
            return 

        self.market_data['profit_and_loss'] = self.pnl_history

        output = ""
        output += "Sandbox logs:\n"
        for i in self.sandbox_logs:
            output += json.dumps(i, indent=2) + "\n"

        output += "\n\n\n\nActivities log:\n"
        market_data_csv = self.market_data.to_csv(index=False, sep=";")
        market_data_csv = market_data_csv.replace("\r\n", "\n")
        output += market_data_csv

        output += "\n\n\n\nTrade History:\n"
        output += json.dumps(self.trades, indent=2)

        with open(filename, 'w') as file:
            file.write(output)

            
    def _add_trades(self, own_trades: Dict[str, List[Trade]], market_trades: Dict[str, List[Trade]]):
        products = set(own_trades.keys()) | set(market_trades.keys()) # Define unified set
        for product in products:
            self.trades.extend([self._trade_to_dict(trade) for trade in own_trades.get(product, [])]) # Creates the list of own trades (Both placed against the market anf against other player's bids)
        for product in products:
            self.trades.extend([self._trade_to_dict(trade) for trade in market_trades.get(product, [])]) # Creates a list of the market trades (includes the ones we participated in + the ones we were not involved in)

    def _trade_to_dict(self, trade: Trade) -> dict[str, Any]: # Self explanatory
        return {
            "timestamp": trade.timestamp,
            "buyer": trade.buyer,
            "seller": trade.seller,
            "symbol": trade.symbol,
            "currency": "SEASHELLS",
            "price": trade.price,
            "quantity": trade.quantity,
        }
        
    def _construct_trading_state(self, traderData, timestamp, listings, order_depths, 
                                 own_trades, market_trades, position, observations):
        state = TradingState(traderData, timestamp, listings, order_depths, 
                             own_trades, market_trades, position, observations)
        return state
    
        
    def _construct_order_depths(self, group):
        order_depths = {} # Dictionary that contains order depths instances for each product given a group (associated to a timestamp)
        for idx, row in group.iterrows(): # Loops through each product in the timestamp
            product = row['product'] # Defines product
            order_depth = OrderDepth() # Initializes an order depth instance 
            for i in range(1, 4): # For each bid/ ask
                if f'bid_price_{i}' in row and f'bid_volume_{i}' in row: # Check 
                    bid_price = row[f'bid_price_{i}'] # Defines bid price
                    bid_volume = row[f'bid_volume_{i}'] # Defines bid volume
                    if not pd.isna(bid_price) and not pd.isna(bid_volume):
                        order_depth.buy_orders[int(bid_price)] = int(bid_volume) # Creates a buy order instance of our order depth class if they are not null
                if f'ask_price_{i}' in row and f'ask_volume_{i}' in row: # Idem for ask
                    ask_price = row[f'ask_price_{i}']
                    ask_volume = row[f'ask_volume_{i}']
                    if not pd.isna(ask_price) and not pd.isna(ask_volume):
                        order_depth.sell_orders[int(ask_price)] = -int(ask_volume) # Changes sign for coherence purposes 
            order_depths[product] = order_depth # Creates the product entry
        return order_depths
    
        
        
    def _execute_buy_order(self, timestamp, order, order_depths, position, cash, trade_history_dict, sandboxLog):
        trades = [] # Empty list containing the trades to be executed
        order_depth = order_depths[order.symbol] # Asks/ bids placed for the product at that timestamp

        for price, volume in list(order_depth.sell_orders.items()): # Loops through the asks
            if price > order.price or order.quantity == 0: # If minimum price is higher than what we are willing to pay  (BECAUSE PRICE(ASK1)<=PRICE(ASK2)<=PRICE(ASK3)!!!), or we do not want to buy this product), exit the loop
                break
            # We can (and will) trade
            trade_volume = min(abs(order.quantity), abs(volume)) # You can't buy more than offered and will only buy as much as you want
            if abs(trade_volume + position[order.symbol]) <= int(self.position_limit[order.symbol]): # Check that we will not be over limit when buying
                trades.append(Trade(order.symbol, price, trade_volume, "SUBMISSION", "", timestamp)) # We save the trade. Buyer's name is SUBMISSION (us)
                position[order.symbol] += trade_volume # Adjust postion details
                self.cash[order.symbol] -= price * trade_volume
                order_depth.sell_orders[price] += trade_volume # Ask's volume adjusted to trade
                order.quantity -= trade_volume # Remaning orders of the product to be placed
            else:
                sandboxLog += f"\nOrders for product {order.symbol} exceeded limit of {self.position_limit[order.symbol]} set" # Returns warning
            

            if order_depth.sell_orders[price] == 0: # If volume is 0, remove instance from market asks
                del order_depth.sell_orders[price]
        
        trades_at_timestamp = trade_history_dict.get(timestamp, []) # Trades executed at this timestamp
        new_trades_at_timestamp = [] # List of trades to be executed
        for trade in trades_at_timestamp: # Loops through timestamp 
            if trade.symbol == order.symbol: # Checks we are comparing the same product
                if trade.price < order.price: # If there was a competitive executed offer in the market
                    trade_volume = min(abs(order.quantity), abs(trade.quantity)) 
                    trades.append(Trade(order.symbol, order.price, trade_volume, "SUBMISSION", "", timestamp)) # We rewrite what happened
                    order.quantity -= trade_volume # Adjust current position. NOTICE THAT PRICE IS SETTLED AT BUYING PRICE!!!!!! (conservative purposes as well)
                    position[order.symbol] += trade_volume
                    self.cash[order.symbol] -= order.price * trade_volume
                    if trade_volume == abs(trade.quantity): # I have bought the total volume of the ask, so my transaction substitutes the other guy's
                        continue
                    else: # Define the new transaction that substitute the other guy's (We believe this is done for simplicity purposes)
                        new_quantity = trade.quantity - trade_volume
                        new_trades_at_timestamp.append(Trade(order.symbol, order.price, new_quantity, "", "", timestamp)) 
                        continue
            new_trades_at_timestamp.append(trade) # If we did not participate in the trade, it does not get modifyed 

        if len(new_trades_at_timestamp) > 0: # If any trades were performed, adjust the transactions book for the timestamp
            trade_history_dict[timestamp] = new_trades_at_timestamp

        return trades, sandboxLog 
        
        
         
    def _execute_sell_order(self, timestamp, order, order_depths, position, cash, trade_history_dict, sandboxLog): # Idém
        trades = []
        order_depth = order_depths[order.symbol]
        
        for price, volume in sorted(order_depth.buy_orders.items(), reverse=True):
            if price < order.price or order.quantity == 0:
                break

            trade_volume = min(abs(order.quantity), abs(volume))
            if abs(position[order.symbol] - trade_volume) <= int(self.position_limit[order.symbol]):
                trades.append(Trade(order.symbol, price, trade_volume, "", "SUBMISSION", timestamp))
                position[order.symbol] -= trade_volume
                self.cash[order.symbol] += price * abs(trade_volume)
                order_depth.buy_orders[price] -= abs(trade_volume)
                order.quantity += trade_volume
            else:
                sandboxLog += f"\nOrders for product {order.symbol} exceeded limit of {self.position_limit[order.symbol]} set"

            if order_depth.buy_orders[price] == 0:
                del order_depth.buy_orders[price]

        trades_at_timestamp = trade_history_dict.get(timestamp, [])
        new_trades_at_timestamp = []
        for trade in trades_at_timestamp:
            if trade.symbol == order.symbol:
                if trade.price > order.price:
                    trade_volume = min(abs(order.quantity), abs(trade.quantity))
                    trades.append(Trade(order.symbol, order.price, trade_volume, "", "SUBMISSION", timestamp))
                    order.quantity += trade_volume
                    position[order.symbol] -= trade_volume
                    self.cash[order.symbol] += order.price * trade_volume
                    if trade_volume == abs(trade.quantity):
                        continue
                    else:
                        new_quantity = trade.quantity - trade_volume
                        new_trades_at_timestamp.append(Trade(order.symbol, order.price, new_quantity, "", "", timestamp))
                        continue
            new_trades_at_timestamp.append(trade)  

        if len(new_trades_at_timestamp) > 0:
            trade_history_dict[timestamp] = new_trades_at_timestamp
                
        return trades, sandboxLog
        
        
        
    def _execute_order(self, timestamp, order, order_depths, position, cash, trades_at_timestamp, sandboxLog):
        if order.quantity == 0: # Trvial scenario
            return []
        
        order_depth = order_depths[order.symbol] # Gets the market bids/ asks of the product at that timestamp
        if order.quantity > 0: # If I want to buy
            return self._execute_buy_order(timestamp, order, order_depths, position, cash, trades_at_timestamp, sandboxLog)
        else: # If I want to sell
            return self._execute_sell_order(timestamp, order, order_depths, position, cash, trades_at_timestamp, sandboxLog)
    
    def _mark_pnl(self, cash, position, order_depths, pnl, product):
        order_depth = order_depths[product]
        
        best_ask = min(order_depth.sell_orders.keys()) # Lowest ask still in the market
        best_bid = max(order_depth.buy_orders.keys()) # Hihjest bid still in the market
        mid = (best_ask + best_bid)/2
        fair = mid # Fair price is defined as average between the best ask and bid, afer having placed our bids
        if product in self.fair_marks: 
            get_fair = self.fair_marks[product] # Recovers fair price calculated prior to simulating our trades
            fair = get_fair(order_depth) # Updates fair price after having placed our trades
        
        pnl[product] = cash[product] + fair * position[product] # Compares what he actually spent (sold) vs what he should have bought (sold) the product for (fair value)
        


# trader

In [ ]:
from datamodel import OrderDepth, UserId, TradingState, Order
from typing import List
import string
import jsonpickle
import numpy as np
import math


class Product: # Defines products
    KELP = "KELP"
    RAINFOREST_RESIN = "RAINFOREST_RESIN"
    SQUID_INK = "SQUID_INK"

# Defines the grid parameter for eeach strategy
PARAMS = {
    Product.KELP: {
        "fair_value": 10000,
        "take_width": 1,
        "clear_width": 0,
        # for making
        "disregard_edge": 1,  # disregards orders for joining or pennying within this value from fair
        "join_edge": 2,  # joins orders within this edge
        "default_edge": 4,
        "soft_position_limit": 10,
    },
    Product.RAINFOREST_RESIN: {
        "take_width": 1,
        "clear_width": -0.25,
        "prevent_adverse": True,
        "adverse_volume": 15,
        "reversion_beta": -0.229,
        "disregard_edge": 1,
        "join_edge": 0,
        "default_edge": 1,
    },
    Product.SQUID_INK: {
        "take_width": 1,
        "clear_width": -0.25,
        "prevent_adverse": True,
        "adverse_volume": 15,
        "reversion_beta": -0.229,
        "disregard_edge": 1,
        "join_edge": 0,
        "default_edge": 1,
    },
}


class Trader:
    def __init__(self, params=None):
        if params is None:
            params = PARAMS # Default initializes parameters
        self.params = params

        self.LIMIT = {Product.KELP: 20, Product.RAINFOREST_RESIN: 20, Product.RAINFOREST_RESIN: 20 } # Defines position limit for each product

    def take_best_orders( # responsible for aggressively taking market orders (i.e., lifting the best available bids or offers from the order book) if the price is favorable compared to a reference value called the fair_value.
        self,
        product: str,
        fair_value: int,
        take_width: float, # minimum profit you want when taking a trade
        orders: List[Order], # List that will store the resulting orders 
        order_depth: OrderDepth,
        position: int,
        buy_order_volume: int, # how much we've bought in this cycle --> output of the function
        sell_order_volume: int, # how much we've sold in this cycle --> output of the function
        prevent_adverse: bool = False, # if enabled, skip trades with too much size
        adverse_volume: int = 0,
    ) -> (int, int):
        position_limit = self.LIMIT[product] # Defines the limit

        if len(order_depth.sell_orders) != 0: # Gets the lowest sell price (best ask) and its volume
            best_ask = min(order_depth.sell_orders.keys())
            best_ask_amount = -1 * order_depth.sell_orders[best_ask]

            if not prevent_adverse or abs(best_ask_amount) <= adverse_volume: # if volume is not too big (or adverse is not enabled)
                if best_ask <= fair_value - take_width: # if The best sell price is lower than our fair value calculated - take width (= is cheap)
                    quantity = min( # The minimum of what is available and what we are allowed to buy (= max amount to buy)
                        best_ask_amount, position_limit - position
                    )  
                    if quantity > 0: # If we can buy
                        orders.append(Order(product, best_ask, quantity)) # send the buy order
                        buy_order_volume += quantity # Update the mmbuy order volume
                        order_depth.sell_orders[best_ask] += quantity # Update the book (ask amount is negative defined)
                        if order_depth.sell_orders[best_ask] == 0: # If the ask's quantity left is 0 then delete it fomr the book
                            del order_depth.sell_orders[best_ask]

        if len(order_depth.buy_orders) != 0: # Idém 
            best_bid = max(order_depth.buy_orders.keys())
            best_bid_amount = order_depth.buy_orders[best_bid]

            if not prevent_adverse or abs(best_bid_amount) <= adverse_volume:
                if best_bid >= fair_value + take_width:
                    quantity = min(
                        best_bid_amount, position_limit + position
                    )  # should be the max we can sell
                    if quantity > 0:
                        orders.append(Order(product, best_bid, -1 * quantity))
                        sell_order_volume += quantity
                        order_depth.buy_orders[best_bid] -= quantity
                        if order_depth.buy_orders[best_bid] == 0:
                            del order_depth.buy_orders[best_bid]

        return buy_order_volume, sell_order_volume

    def market_make( # This function is used to place passive buy and sell orders (i.e., you’re not taking market orders — you're letting others come to you).
        self,
        product: str,
        orders: List[Order], # List to store the orders
        bid: int, # Price we want to buy at
        ask: int,# Price we want to sell at
        position: int, # current inventory
        buy_order_volume: int, # How much we've already commited to buying --> output of the function
        sell_order_volume: int,# How much we've already commited to selling --> output of the function
    ) -> (int, int):
        buy_quantity = self.LIMIT[product] - (position + buy_order_volume) # defines how much we have left to buy (sell)
        if buy_quantity > 0:
            orders.append(Order(product, round(bid), buy_quantity))  # Defines an order instance and appends it to the list

        sell_quantity = self.LIMIT[product] + (position - sell_order_volume)
        if sell_quantity > 0:
            orders.append(Order(product, round(ask), -sell_quantity))  # Defines an order instance and appends it to the list
        return buy_order_volume, sell_order_volume

    def clear_position_order( # This functions assesses risk management and position cleanup when your inventory gets too far from zero.
        self, # Every variable has previously been defined
        product: str,
        fair_value: float,
        width: int,
        orders: List[Order],
        order_depth: OrderDepth,
        position: int,
        buy_order_volume: int,
        sell_order_volume: int,
    ) -> List[Order]:
        position_after_take = position + buy_order_volume - sell_order_volume # first we define our current position on the product
        fair_for_bid = round(fair_value - width) # Threshold that defines maximum price willing to pay 
        fair_for_ask = round(fair_value + width)# Threshold that defines minimum price willing to sell to

        buy_quantity = self.LIMIT[product] - (position + buy_order_volume) # Only applies when > 0
        sell_quantity = self.LIMIT[product] + (position - sell_order_volume)# Only applies when > 0

        if position_after_take > 0: # If we're too long
            # Aggregate volume from all buy orders with price greater than fair_for_ask
            clear_quantity = sum( # Count of how many buy orders are out there above our fair_for_ask ( number of favorable prices for us to sell to)
                volume
                for price, volume in order_depth.buy_orders.items()
                if price >= fair_for_ask
            )
            clear_quantity = min(clear_quantity, position_after_take) # we can't sell more than we are allowed to and we can't sell more than our position
            sent_quantity = min(sell_quantity, clear_quantity)
            if sent_quantity > 0:
                orders.append(Order(product, fair_for_ask, -abs(sent_quantity))) # Fire a marketable limit order to sell and reduce exposure.
                sell_order_volume += abs(sent_quantity) # update our sale order volume

        if position_after_take < 0: # Idém (If we're too short) 
            # Aggregate volume from all sell orders with price lower than fair_for_bid
            clear_quantity = sum(
                abs(volume)
                for price, volume in order_depth.sell_orders.items()
                if price <= fair_for_bid
            )
            clear_quantity = min(clear_quantity, abs(position_after_take))
            sent_quantity = min(buy_quantity, clear_quantity)
            if sent_quantity > 0:
                orders.append(Order(product, fair_for_bid, abs(sent_quantity)))
                buy_order_volume += abs(sent_quantity)

        return buy_order_volume, sell_order_volume

    def rainforestresin_fair_value(self, order_depth: OrderDepth, traderObject) -> float: # This function calculates a dynamic fair value for STARFRUIT, as it uses best bids/ asks, order book liquidity and mean reversion logic
        if len(order_depth.sell_orders) != 0 and len(order_depth.buy_orders) != 0: # If there are available buy and sell orders
            best_ask = min(order_depth.sell_orders.keys()) # Gets the best sell price in the market
            best_bid = max(order_depth.buy_orders.keys())# Gets the best buy price in the market
            filtered_ask = [ # Filters out small-volume orders, only considers orders with volume above threshold (liquid)
                price
                for price in order_depth.sell_orders.keys()
                if abs(order_depth.sell_orders[price])
                >= self.params[Product.RAINFOREST_RESIN]["adverse_volume"]
            ]
            filtered_bid = [ # Idém
                price
                for price in order_depth.buy_orders.keys()
                if abs(order_depth.buy_orders[price])
                >= self.params[Product.RAINFOREST_RESIN]["adverse_volume"]
            ]
            mm_ask = min(filtered_ask) if len(filtered_ask) > 0 else None # Calculates again best ask
            mm_bid = max(filtered_bid) if len(filtered_bid) > 0 else None # Calculates again best bid
            if mm_ask == None or mm_bid == None: 
                if traderObject.get("rainforestresin_last_price", None) == None:
                    mmmid_price = (best_ask + best_bid) / 2 # Fallback to the best bid/ ask midpoint
                else:
                    mmmid_price = traderObject["rainforestresin_last_price"] # or the last known price (if available),
            else:
                mmmid_price = (mm_ask + mm_bid) / 2 

            if traderObject.get("rainforestresin_last_price", None) != None: # If we have a last price
                last_price = traderObject["rainforestresin_last_price"] 
                last_returns = (mmmid_price - last_price) / last_price # How much the price has changed since
                pred_returns = (
                    last_returns * self.params[Product.RAINFOREST_RESIN]["reversion_beta"] # This coefficient tells how much this product tend to revert back
                )
                fair = mmmid_price + (mmmid_price * pred_returns) # Fair price is mid price times how much we expect it to return to its last price
            else:
                fair = mmmid_price
            traderObject["rainforestresin_last_price"] = mmmid_price # store the new last price
            return fair # return fair price
        return None# return None, no fair price was calculated


    def take_orders( # This function actively tries to take existing orders from the market if the price is good enough — aka market taker behavior.
        self,
        product: str,
        order_depth: OrderDepth,
        fair_value: float,
        take_width: float,
        position: int,
        prevent_adverse: bool = False,
        adverse_volume: int = 0,
    ) -> (List[Order], int, int):
        orders: List[Order] = []
        buy_order_volume = 0
        sell_order_volume = 0

        buy_order_volume, sell_order_volume = self.take_best_orders(
            product,
            fair_value,
            take_width,
            orders,
            order_depth,
            position,
            buy_order_volume,
            sell_order_volume,
            prevent_adverse,
            adverse_volume,
        )
        return orders, buy_order_volume, sell_order_volume

    def clear_orders(
        self,
        product: str,
        order_depth: OrderDepth,
        fair_value: float,
        clear_width: int,
        position: int,
        buy_order_volume: int,
        sell_order_volume: int,
    ) -> (List[Order], int, int):
        orders: List[Order] = []
        buy_order_volume, sell_order_volume = self.clear_position_order(
            product,
            fair_value,
            clear_width,
            orders,
            order_depth,
            position,
            buy_order_volume,
            sell_order_volume,
        )
        return orders, buy_order_volume, sell_order_volume

    def make_orders(
        #The make_orders function is responsible for “making” liquidity on the order book. Unlike taking orders from the existing order book,
        #market making involves placing new limit orders into the market to provide liquidity. This function determines at what prices and 
        #sizes the new orders should be placed by “pennying” (i.e., improving the price) or “joining” existing orders.   

        self,
        product, 
        order_depth: OrderDepth, 
        fair_value: float,
        position: int, 
        buy_order_volume: int,
        sell_order_volume: int,
        disregard_edge: float,  # A price distance from fair value within which you ignore existing orders for the purpose of setting your new order. 
                                # This prevents your new orders from interacting with orders that are too close to the current fair value.

        join_edge: float,  # A threshold that, if met, encourages your strategy to “join” an existing order rather than undercut (or “penny”) it.
        default_edge: float,  # default edge to request if there are no levels to penny or join
        manage_position: bool = False,
        soft_position_limit: int = 0,
        # will penny all other levels with higher edge
        # manage_position (optional) and soft_position_limit (optional): 
        # These allow for an adjustment to the bid or ask based on your current net position. If your inventory is too high or too low relative to 
        # a soft position limit, the prices of your orders are adjusted to make it more likely that your orders will help move your inventory back toward the target.
    ):

        orders: List[Order] = []
        asks_above_fair = [ # Gets all of the favorable asks (sell orders that are above our fair value + the edge)
            price
            for price in order_depth.sell_orders.keys()
            if price > fair_value + disregard_edge
        ]
        bids_below_fair = [ # Gets all of the favorable bids (buy orders that are below our fair value + the edge)
            price
            for price in order_depth.buy_orders.keys()
            if price < fair_value - disregard_edge
        ]

        best_ask_above_fair = min(asks_above_fair) if len(asks_above_fair) > 0 else None # Gets the best (lowest) ask among the favorable ones
        best_bid_below_fair = max(bids_below_fair) if len(bids_below_fair) > 0 else None # Gets the best (highest) bid among the favorable ones

        ask = round(fair_value + default_edge)
        if best_ask_above_fair != None: # If there is at least one favorable ask
            if abs(best_ask_above_fair - fair_value) <= join_edge: # We check if the market ask is close enough to our fair value to join the order
                ask = best_ask_above_fair  # Join
            else: # If the gap is bigger than the join threshold, we penny the order by subtracting 1 tick
                ask = best_ask_above_fair - 1  # Penny

        bid = round(fair_value - default_edge) # Idém 
        if best_bid_below_fair != None:
            if abs(fair_value - best_bid_below_fair) <= join_edge:
                bid = best_bid_below_fair
            else:
                bid = best_bid_below_fair + 1

        if manage_position: # If manage_position is enabled, the bid/ask prices are further adjusted:
            if position > soft_position_limit: # If too long the ask price is reduced by one tick to encourage selling.
                ask -= 1
            elif position < -1 * soft_position_limit: # If too short the bid price is increased by one tick to encourage buying.
                bid += 1

        buy_order_volume, sell_order_volume = self.market_make( # Market make using the bid and ask prices just calculated
            product,
            orders,
            bid,
            ask,
            position,
            buy_order_volume,
            sell_order_volume,
        )

        return orders, buy_order_volume, sell_order_volume
    
    # New method: Mean Reversion Strategy for SQUID_INK using Exponential Moving Average (EMA)
    def mean_reversion_squid_ink(self, order_depth: OrderDepth, traderObject, position: int) -> (List[Order], int, int):
        orders: List[Order] = []
        buy_order_volume = 0
        sell_order_volume = 0

        # Check that there are both buy and sell orders
        if len(order_depth.sell_orders) == 0 or len(order_depth.buy_orders) == 0:
            return orders, buy_order_volume, sell_order_volume

        # Calculate current mid-price
        best_ask = min(order_depth.sell_orders.keys())
        best_bid = max(order_depth.buy_orders.keys())
        mid_price = (best_ask + best_bid) / 2

        # Retrieve or initialize price history for volatility calculation
        price_history = traderObject.get("SQUID_INK_prices", [])
        price_history.append(mid_price)
        history_length = self.params[Product.SQUID_INK].get("price_history_length", 20)
        if len(price_history) > history_length:
            price_history = price_history[-history_length:]
        traderObject["SQUID_INK_prices"] = price_history

        # Compute Exponential Moving Average (EMA)
        # Smoothing factor alpha (e.g., 0.1 gives more weight to recent prices)
        ema_alpha = self.params[Product.SQUID_INK].get("ema_alpha", 0.1)
        if "SQUID_INK_ema" not in traderObject:
            ema = mid_price  # Initialize EMA with current mid price
        else:
            prev_ema = traderObject["SQUID_INK_ema"]
            ema = ema_alpha * mid_price + (1 - ema_alpha) * prev_ema
        traderObject["SQUID_INK_ema"] = ema

        # For volatility, we still compute a simple standard deviation over the history.
        std_dev = np.std(price_history)
        zscore = (mid_price - ema) / std_dev if std_dev > 0 else 0.0

        # Use threshold from parameters
        threshold = self.params[Product.SQUID_INK].get("mean_reversion_threshold", 1.5)
        position_limit = self.LIMIT[Product.SQUID_INK]

        # If the price is significantly below the EMA, generate a buy signal.
        if zscore < -threshold:
            quantity = min(position_limit - position, int(abs(zscore) * self.params[Product.SQUID_INK].get("order_scale", 10)))
            if quantity > 0:
                orders.append(Order(Product.SQUID_INK, round(mid_price), quantity))
                buy_order_volume += quantity

        # If the price is significantly above the EMA, generate a sell signal.
        elif zscore > threshold:
            quantity = min(position_limit + position, int(abs(zscore) * self.params[Product.SQUID_INK].get("order_scale", 10)))
            if quantity > 0:
                orders.append(Order(Product.SQUID_INK, round(mid_price), -quantity))
                sell_order_volume += quantity

        return orders, buy_order_volume, sell_order_volume

    def run(self, state: TradingState):
        traderObject = {}
        if state.traderData != None and state.traderData != "":
            traderObject = jsonpickle.decode(state.traderData)

        result = {}

        if Product.KELP in self.params and Product.KELP in state.order_depths:
            kelp_position = (  # Check your current position (inventory) in the product.If you haven't traded this product yet, your position is assumed to be 0.
                state.position[Product.KELP]
                if Product.KELP in state.position
                else 0
            )
            kelp_take_orders, buy_order_volume, sell_order_volume = ( # Call your take orders with the logic defined in the parameters
                self.take_orders(
                    Product.KELP,
                    state.order_depths[Product.KELP],
                    self.params[Product.KELP]["fair_value"],
                    self.params[Product.KELP]["take_width"],
                    kelp_position,
                )
            )
            kelp_clear_orders, buy_order_volume, sell_order_volume = ( # Call your clear orders (netting positions) with the logic defined in the parameters
                self.clear_orders(
                    Product.KELP,
                    state.order_depths[Product.KELP],
                    self.params[Product.KELP]["fair_value"],
                    self.params[Product.KELP]["clear_width"],
                    kelp_position,
                    buy_order_volume, # VOLUME UPDATED AFTER CALLING TAKE_ORDERS!!!
                    sell_order_volume, # VOLUME UPDATED AFTER CALLING TAKE_ORDERS!!!
                )
            )
            kelp_make_orders, _, _ = self.make_orders( # Call your make orders (Market Making) with the logic defined in the parameters
                Product.KELP,
                state.order_depths[Product.KELP],
                self.params[Product.KELP]["fair_value"],
                kelp_position,
                buy_order_volume, # VOLUME UPDATED AFTER CALLING CLEAR_ORDERS!!!
                sell_order_volume, # VOLUME UPDATED AFTER CALLING CLEAR_ORDERS!!!
                self.params[Product.KELP]["disregard_edge"],
                self.params[Product.KELP]["join_edge"],
                self.params[Product.KELP]["default_edge"],
                True,
                self.params[Product.KELP]["soft_position_limit"],
            )
            result[Product.KELP] = ( # Append all of your orders to the result
                kelp_take_orders + kelp_clear_orders + kelp_make_orders
            )

        if Product.RAINFOREST_RESIN in self.params and Product.RAINFOREST_RESIN in state.order_depths: # Idém
            rainforestresin_position = (
                state.position[Product.RAINFOREST_RESIN]
                if Product.RAINFOREST_RESIN in state.position
                else 0
            )
            rainforestresin_fair_value = self.rainforestresin_fair_value( # Calculate RAINFOREST_RESIN fair value
                state.order_depths[Product.RAINFOREST_RESIN], traderObject
            )
            rainforestresin_take_orders, buy_order_volume, sell_order_volume = ( # Here we include adverse volume to limit risk
                self.take_orders(
                    Product.RAINFOREST_RESIN,
                    state.order_depths[Product.RAINFOREST_RESIN],
                    rainforestresin_fair_value,
                    self.params[Product.RAINFOREST_RESIN]["take_width"],
                    rainforestresin_position,
                    self.params[Product.RAINFOREST_RESIN]["prevent_adverse"],
                    self.params[Product.RAINFOREST_RESIN]["adverse_volume"],
                )
            )
            rainforestresin_clear_orders, buy_order_volume, sell_order_volume = (
                self.clear_orders(
                    Product.RAINFOREST_RESIN,
                    state.order_depths[Product.RAINFOREST_RESIN],
                    rainforestresin_fair_value,
                    self.params[Product.RAINFOREST_RESIN]["clear_width"],
                    rainforestresin_position,
                    buy_order_volume,
                    sell_order_volume,
                )
            )
            rainforestresin_make_orders, _, _ = self.make_orders( # Here we do not include soft edge
                Product.RAINFOREST_RESIN,
                state.order_depths[Product.RAINFOREST_RESIN],
                rainforestresin_fair_value,
                rainforestresin_position,
                buy_order_volume,
                sell_order_volume,
                self.params[Product.RAINFOREST_RESIN]["disregard_edge"],
                self.params[Product.RAINFOREST_RESIN]["join_edge"],
                self.params[Product.RAINFOREST_RESIN]["default_edge"],
            )
            result[Product.RAINFOREST_RESIN] = (
                rainforestresin_take_orders + rainforestresin_clear_orders + rainforestresin_make_orders
            )

         # Integrate the new exponential mean reversion strategy for SQUID_INK.
        if Product.SQUID_INK in self.params and Product.SQUID_INK in state.order_depths:
            squid_position = state.position.get(Product.SQUID_INK, 0)
            squid_orders, _, _ = self.mean_reversion_squid_ink(
                state.order_depths[Product.SQUID_INK],
                traderObject,
                squid_position,
            )
            result[Product.SQUID_INK] = squid_orders

        conversions = 1
        traderData = jsonpickle.encode(traderObject)

        return result, conversions, traderData


# backtest run

In [6]:
from datamodel import Listing, Observation
import pandas as pd
import json
import io

In [82]:
def _process_data_(file):
    with open(file, 'r') as file:
        log_content = file.read()
    sections = log_content.split('Sandbox logs:')[1].split('Activities log:')
    sandbox_log =  sections[0].strip()
    activities_log = sections[1].split('Trade History:')[0]
    # sandbox_log_list = [json.loads(line) for line in sandbox_log.split('\n')]
    trade_history =  json.loads(sections[1].split('Trade History:')[1])
    # sandbox_log_df = pd.DataFrame(sandbox_log_list)
    market_data_df = pd.read_csv(io.StringIO(activities_log), sep=";", header=0)
    trade_history_df = pd.json_normalize(trade_history)
    return market_data_df, trade_history_df

In [83]:
# from round_1_parameterized_v3 import Trader
# from round_1_cleaned_v2 import Trader

### setup

In [25]:
def calculate_rainforest_fair(order_depth):
    # assumes order_depth has orders in it 
    best_ask = min(order_depth.sell_orders.keys())
    best_bid = max(order_depth.buy_orders.keys())
    filtered_ask = [price for price in order_depth.sell_orders.keys() if abs(order_depth.sell_orders[price]) >= 15]
    filtered_bid = [price for price in order_depth.buy_orders.keys() if abs(order_depth.buy_orders[price]) >= 15]
    mm_ask = min(filtered_ask) if len(filtered_ask) > 0 else best_ask
    mm_bid = max(filtered_bid) if len(filtered_bid) > 0 else best_bid

    mmmid_price = (mm_ask + mm_bid) / 2
    return mmmid_price
    
def calculate_kelp_fair(order_depth):
    return 10000

In [85]:
listings = {
    'KELP': Listing(symbol='KELP', product='KELP', denomination='SEASHELLS'),
    'RAINFOREST_RESIN': Listing(symbol='RAINFOREST_RESIN', product='RAINFOREST_RESIN', denomination='SEASHELLS')
}

position_limit = {
    'KELP': 50,
    'RAINFOREST_RESIN': 50
}

fair_calculations = {
    "KELP": calculate_rainforest_fair,
    "RAINFOREST_RESIN": calculate_kelp_fair
}


In [86]:
# with fair prediction
day = 0
market_data = pd.read_csv(rf"C:\Users\ramon\OneDrive\Desktop\Python projects\Trading\Prosperity 3\Data\Round 1\round-1-island-data-bottle\prices_round_1_day_{day}.csv", sep=";", header=0)
trade_history = pd.read_csv(rf"C:\Users\ramon\OneDrive\Desktop\Python projects\Trading\Prosperity 3\Data\Round 1\round-1-island-data-bottle\trades_round_1_day_{day}.csv", sep=";", header=0)

market_data = market_data[market_data['product']!='SQUID_INK']
trade_history = trade_history[trade_history['symbol']!='SQUID_INK']

trader = Trader()
path = r'C:\Users\ramon\OneDrive\Desktop\Python projects\Trading\Prosperity 3\volume.log'
backtester = Backtester(trader, listings, position_limit, fair_calculations, market_data, trade_history, path)
backtester.run()
print(backtester.pnl)

TypeError: Object of type OrderDepth is not JSON serializable

In [51]:
# with fair prediction
day = 0
market_data = pd.read_csv(rf"C:\Users\ramon\OneDrive\Desktop\Python projects\Trading\Prosperity 3\Data\Round 1\round-1-island-data-bottle\prices_round_1_day_{day}.csv", sep=";", header=0)
trade_history = pd.read_csv(rf"C:\Users\ramon\OneDrive\Desktop\Python projects\Trading\Prosperity 3\Data\Round 1\round-1-island-data-bottle\trades_round_1_day_{day}.csv", sep=";", header=0)

market_data = market_data[market_data['product']!='SQUID_INK']
trade_history = trade_history[trade_history['symbol']!='SQUID_INK']

trader = Trader()
path = r'C:\Users\ramon\OneDrive\Desktop\Python projects\Trading\Prosperity 3\volume.log'
backtester = Backtester(trader, listings, position_limit, fair_calculations, market_data, trade_history, path)
backtester.run()
print(backtester.pnl)


{'KELP': 6228057.0, 'RAINFOREST_RESIN': -78}


`{'AMETHYSTS': 14554, 'RAINFOREST_RESIN': 14144.5}`

In [135]:
day = 0
market_data = pd.read_csv(f"./round-1-island-data-bottle/prices_round_1_day_{day}.csv", sep=";", header=0)
trade_history = pd.read_csv(f"./round-1-island-data-bottle/trades_round_1_day_{day}_nn.csv", sep=";", header=0)

trader = Trader()
backtester = Backtester(trader, listings, position_limit, market_data, trade_history, "trade_history_sim.log")
backtester.run()
print(backtester.pnl)

{'AMETHYSTS': 14554.0, 'STARFRUIT': 14118.0}


# backtest gridsearch

In [7]:
import itertools

def generate_param_combinations(param_grid):
    param_names = param_grid.keys()
    param_values = param_grid.values()
    combinations = list(itertools.product(*param_values))
    return [dict(zip(param_names, combination)) for combination in combinations]

In [8]:
import os
from tqdm import tqdm

def run_backtests(trader, listings, position_limit, fair_calcs, market_data, trade_history, backtest_dir, param_grid, symbol):
    if not os.path.exists(backtest_dir):
        os.makedirs(backtest_dir)

    param_combinations = generate_param_combinations(param_grid[symbol])

    results = []
    for params in tqdm(param_combinations, desc=f"Running backtests for {symbol}", unit="backtest"):
        trader.params = {symbol: params}
        backtester = Backtester(trader, listings, position_limit, fair_calcs, market_data, trade_history)
        backtester.run()

        param_str = "-".join([f"{key}={value}" for key, value in params.items()])
        log_filename = f"{backtest_dir}/{symbol}_{param_str}.log"
        backtester._log_trades(log_filename)

        results.append((params, backtester.pnl[symbol]))

    return results

### setup

In [27]:
listings = {
    'KELP': Listing(symbol='KELP', product='KELP', denomination='SEASHELLS'),
    'RAINFOREST_RESIN': Listing(symbol='RAINFOREST_RESIN', product='RAINFOREST_RESIN', denomination='SEASHELLS')
}

position_limit = {
    'KELP': 50,
    'RAINFOREST_RESIN': 50
}

fair_calculations = {
    "KELP": calculate_kelp_fair,
    "RAINFOREST_RESIN": calculate_rainforest_fair
}


In [28]:
day = 0
#market_data = pd.read_csv(rf"C:\Users\ramon\OneDrive\Desktop\Python projects\Trading\Prosperity 3\Data\Round 1\round-1-island-data-bottle\prices_round_1_day_{day}.csv", sep=";", header=0)
#trade_history = pd.read_csv(rf"C:\Users\ramon\OneDrive\Desktop\Python projects\Trading\Prosperity 3\Data\Round 1\round-1-island-data-bottle\trades_round_1_day_{day}.csv", sep=";", header=0)

market_data = pd.read_csv(rf"C:\Users\gonzaal\Desktop\Personal\Personal python projects\Prosperity 3\round-1-island-data-bottle\round-1-island-data-bottle\prices_round_1_day_{day}.csv", sep=";", header=0)
trade_history = pd.read_csv(rf"C:\Users\gonzaal\Desktop\Personal\Personal python projects\Prosperity 3\round-1-island-data-bottle\round-1-island-data-bottle\trades_round_1_day_{day}.csv", sep=";", header=0)

market_data = market_data[market_data['product']!='SQUID_INK']
trade_history = trade_history[trade_history['symbol']!='SQUID_INK']

In [18]:
trade_history

,timestamp,buyer,seller,symbol,currency,price,quantity
0,100,NaN,NaN,RAINFOREST_RESIN,SEASHELLS,10002.0,1
1,300,NaN,NaN,KELP,SEASHELLS,2029.0,6
2,300,NaN,NaN,RAINFOREST_RESIN,SEASHELLS,9998.0,2
3,300,NaN,NaN,RAINFOREST_RESIN,SEASHELLS,9998.0,1
4,300,NaN,NaN,SQUID_INK,SEASHELLS,1965.0,6
...,...,...,...,...,...,...,...
8177,999500,NaN,NaN,KELP,SEASHELLS,2032.0,2
8178,999500,NaN,NaN,RAINFOREST_RESIN,SEASHELLS,9995.0,2
8179,999500,NaN,NaN,SQUID_INK,SEASHELLS,1830.0,2
8180,999600,NaN,NaN,KELP,SEASHELLS,2032.0,1


### run

In [29]:
backtest_dir = "backtest_test_clear_width"

param_grid = {
    Product.KELP: {
        "fair_value": [10000],
        "take_width": [1],
        "clear_width": [0.5],
        "volume_limit": [0],
        # for making
        "disregard_edge": [1],  # disregards orders for joining or pennying within this value from fair
        "join_edge": [2],# joins orders within this edge 
        "default_edge": [4],
        "soft_position_limit": [30,40]
    },
    Product.RAINFOREST_RESIN: {
        "take_width": [1],
        "clear_width": [0, -0.25],
        "prevent_adverse": [True],
        "adverse_volume": [15],
        "reversion_beta": [-0.229],
        # for making
        "disregard_edge": [1],
        "join_edge": [3],
        "default_edge": [5],
    },
}



trader = Trader()

kelp_results = run_backtests(trader, listings, position_limit, fair_calculations, market_data, trade_history, backtest_dir, param_grid, "KELP")
print("KELP results:")
for params, pnl in kelp_results: 
    print(params)
    print(f"pnl: {pnl}")
    print("="*80)

starfruit_results = run_backtests(trader, listings, position_limit, fair_calculations, market_data, trade_history, backtest_dir, param_grid, "RAINFOREST_RESIN")
print("STARFRUIT results:")
for params, pnl in starfruit_results: 
    print(params)
    print(f"pnl: {pnl}")
    print("="*80)

Running backtests for KELP: 100%|██████████| 2/2 [01:41<00:00, 50.52s/backtest]


KELP results:
{'fair_value': 10000, 'take_width': 1, 'clear_width': 0.5, 'volume_limit': 0, 'disregard_edge': 1, 'join_edge': 2, 'default_edge': 4, 'soft_position_limit': 30}
pnl: 5702832
{'fair_value': 10000, 'take_width': 1, 'clear_width': 0.5, 'volume_limit': 0, 'disregard_edge': 1, 'join_edge': 2, 'default_edge': 4, 'soft_position_limit': 40}
pnl: 5702832


Running backtests for RAINFOREST_RESIN: 100%|██████████| 2/2 [00:56<00:00, 28.23s/backtest]

STARFRUIT results:
{'take_width': 1, 'clear_width': 0, 'prevent_adverse': True, 'adverse_volume': 15, 'reversion_beta': -0.229, 'disregard_edge': 1, 'join_edge': 3, 'default_edge': 5}
pnl: 79675.0
{'take_width': 1, 'clear_width': -0.25, 'prevent_adverse': True, 'adverse_volume': 15, 'reversion_beta': -0.229, 'disregard_edge': 1, 'join_edge': 3, 'default_edge': 5}
pnl: 79657.0


## analyze

In [30]:

def analyze_log_files(backtest_dir):
    log_files = [f for f in os.listdir(backtest_dir) if f.endswith('.log')]
    
    results = []
    for log_file in log_files:
        file_path = os.path.join(backtest_dir, log_file)
        
        # Extract symbol and parameters from the file name
        file_name = os.path.splitext(log_file)[0]
        print(file_name)
        symbol, params_str = file_name.split('-', 1)
        params = dict(param.split('=') for param in params_str.split('-'))
        
        # Read the contents of the log file
        with open(file_path, 'r') as file:
            log_content = file.read()
        
        # Store the symbol, parameters, and log content in the results
        results.append({
            'symbol': symbol,
            'params': params,
            'log_content': log_content
        })
    
    return results

# Analyze the log files
log_analysis_results = analyze_log_files(backtest_dir)

# Print the results
for result in log_analysis_results:
    print(f"Symbol: {result['symbol']}")
    print(f"Parameters: {result['params']}")
#     print(f"Log Content:\n{result['log_content']}\n")

AMETHYSTS_fair_value=10000_take_width=3_clear_width=0_volume_limit=15


ValueError: dictionary update sequence element #0 has length 1; 2 is required

In [236]:
sorted_starfruit_results = sorted(starfruit_results, key=lambda x: x[1], reverse=True)


In [239]:
sorted_starfruit_results[0:100]

[({'take_width': 1,
   'clear_width': -0.25,
   'prevent_adverse': True,
   'adverse_volume': 15,
   'reversion_beta': -0.229,
   'disregard_edge': 1,
   'join_edge': 0,
   'default_edge': 1},
  14808.0),
 ({'take_width': 1,
   'clear_width': -0.25,
   'prevent_adverse': True,
   'adverse_volume': 15,
   'reversion_beta': -0.229,
   'disregard_edge': 1,
   'join_edge': 0,
   'default_edge': 2},
  14808.0),
 ({'take_width': 1,
   'clear_width': -0.25,
   'prevent_adverse': True,
   'adverse_volume': 15,
   'reversion_beta': -0.229,
   'disregard_edge': 1,
   'join_edge': 0,
   'default_edge': 3},
  14808.0),
 ({'take_width': 1,
   'clear_width': -0.25,
   'prevent_adverse': True,
   'adverse_volume': 15,
   'reversion_beta': -0.229,
   'disregard_edge': 1,
   'join_edge': 0,
   'default_edge': 4},
  14808.0),
 ({'take_width': 1,
   'clear_width': -0.25,
   'prevent_adverse': True,
   'adverse_volume': 15,
   'reversion_beta': -0.229,
   'disregard_edge': 1,
   'join_edge': 0,
   'defaul